* [ArXiv](https://arxiv.org/pdf/2312.01020)
* [GitHub](https://github.com/Yuanzhe-Jia/ResNLS)

In [1]:
import os
import mlflow
from mlflow.models import infer_signature
import seaborn as sns

import math
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics

In [ ]:
sns.set_theme("paper", rc={"figure.figsize": (16, 4)})

mlflow.set_experiment("Stock Market Predictions")
mlflow.start_run(run_name='ResNLS')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MlflowException: Cannot set a deleted experiment 'Stock Market Predictions' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.

In [ ]:
df = pd.read_csv("../data/SSEC.csv")
data = pd.DataFrame(pd.to_numeric(df["Close"]))
dataset = np.reshape(data.values, (df.shape[0], 1))

# normalise the dataset
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
def split_data(dataset, train_day, predict_day):
    x = []
    y = []
    for i in range(train_day, len(dataset)-predict_day+1):
        x.append(dataset[i-train_day : i, 0])
        y.append(dataset[i+predict_day-1, 0])
    return x, y

# x => data from previous days; y => data in the next day
def reshape_data(train_data, test_data, days):
    x_train, y_train = split_data(train_data, days, 1)
    x_test, y_test = split_data(test_data, days, 1)
    # convert data into numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_test, y_test = np.array(x_test), np.array(y_test)
    # reshape the data for neural network training
    x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
    x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
    return x_train, y_train, x_test, y_test

# create the scaled training data set
training_data_len = math.ceil(len(dataset) * 0.9087)
train_data = scaled_data[0:training_data_len, :]
#print(data[:train_data.shape[0]].tail())

# create the scaled test data set
test_data = scaled_data[training_data_len-5: , :]

# use 5 consecutive trading days as the unit step size sliding through the stock price data
x_train_5, y_train_5, x_test_5, y_test_5 = reshape_data(train_data, test_data, 5)
print("when sequence length is 5, data shape:", x_train_5.shape, y_train_5.shape, x_test_5.shape, y_test_5.shape)

when sequence length is 5, data shape: (3088, 1, 5) (3088,) (310, 1, 5) (310,)


In [ ]:
n_input = 5; n_hidden = 64

class ResNLS(nn.Module):

    def __init__(self):
        super(ResNLS, self).__init__()

        # intialise weights of the attention mechanism
        self.weight = nn.Parameter(torch.zeros(1)).to(device)

        # intialise cnn structure
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=n_hidden, kernel_size=3, stride=1, padding=1), # ((5 + 1*2 - 3)/1 + 1) = 5
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(n_hidden, eps=1e-5),
            nn.Dropout(0.1),

            nn.Conv1d(in_channels=n_hidden, out_channels=n_hidden, kernel_size=3, stride=1, padding=1), # ((5 + 1*2 - 3)/1 + 1) = 5
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(n_hidden, eps=1e-5),

            nn.Flatten(),
            nn.Linear(n_input * n_hidden, n_input)
        )

        # intialise lstm structure
        self.lstm = nn.LSTM(n_input, n_hidden, batch_first=True, bidirectional=False)
        self.linear = nn.Linear(n_hidden, 1)


    def forward(self, x):

        cnn_output = self.cnn(x)
        cnn_output = cnn_output.view(-1, 1, n_input)

        residuals = x + self.weight * cnn_output

        _, (h_n, _)  = self.lstm(x)
        y_hat = self.linear(h_n[0,:,:])

        return y_hat

In [ ]:
##################### model training #####################

# prepare validation data
val_input = torch.tensor(x_test_5, dtype=torch.float).to(device)
val_target = torch.tensor(y_test_5, dtype=torch.float).to(device)

# initialization
epochs = 50; batch_size = 64
learning_rate = 1e-3

# model instance
model = ResNLS().to(device)

# loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# mini-batch training
if x_train_5.shape[0] % batch_size == 0:
    batch_num = int(x_train_5.shape[0] / batch_size)
else:
    batch_num = int(x_train_5.shape[0] / batch_size) + 1

params = {
    "num_epochs": epochs,
    "batch_size": batch_size,
    "learning rate": learning_rate,
    "objective": type(criterion).__name__,
    "optimizer": type(optimizer).__name__,
    "hidden size": n_hidden,
    "sequence length": n_input 
}
mlflow.log_params(params)

for epoch in range(epochs):
    for j in range(batch_num):

        # prepare training data
        train_input = torch.tensor(x_train_5[j * batch_size : (j+1) * batch_size], dtype=torch.float).to(device)
        train_targe = torch.tensor(y_train_5[j * batch_size : (j+1) * batch_size], dtype=torch.float).to(device)

        # training
        model.train()
        optimizer.zero_grad()
        train_output = model(train_input)
        train_loss = criterion(train_output, train_targe.unsqueeze(-1))
        train_loss.backward()
        optimizer.step()
        mlflow.log_metric("train loss", train_loss.item(), step=epoch+1)
                  
    if (epoch+1) % (epochs/20) == 0:
        with torch.no_grad():
            model.eval()
            val_output = model(val_input)
            val_loss = criterion(val_output, val_target.unsqueeze(-1))   
            mlflow.log_metric("val loss", val_loss.item(), step=epoch+1)        
            print("Epoch: {:>3}, train loss: {:.4f}, val loss: {:.4f}".format(epoch+1, train_loss.item(), val_loss.item()))

Epoch:  50, train loss: 0.0002, val loss: 0.0002
Epoch: 100, train loss: 0.0001, val loss: 0.0001
Epoch: 150, train loss: 0.0001, val loss: 0.0001
Epoch: 200, train loss: 0.0001, val loss: 0.0001
Epoch: 250, train loss: 0.0001, val loss: 0.0001
Epoch: 300, train loss: 0.0001, val loss: 0.0001
Epoch: 350, train loss: 0.0001, val loss: 0.0001
Epoch: 400, train loss: 0.0001, val loss: 0.0001
Epoch: 450, train loss: 0.0001, val loss: 0.0001
Epoch: 500, train loss: 0.0001, val loss: 0.0001
Epoch: 550, train loss: 0.0001, val loss: 0.0001
Epoch: 600, train loss: 0.0001, val loss: 0.0001
Epoch: 650, train loss: 0.0001, val loss: 0.0001
Epoch: 700, train loss: 0.0001, val loss: 0.0001
Epoch: 750, train loss: 0.0001, val loss: 0.0001
Epoch: 800, train loss: 0.0001, val loss: 0.0001
Epoch: 850, train loss: 0.0001, val loss: 0.0001
Epoch: 900, train loss: 0.0001, val loss: 0.0001
Epoch: 950, train loss: 0.0001, val loss: 0.0001
Epoch: 1000, train loss: 0.0001, val loss: 0.0001


In [ ]:
##################### model validation #####################

# get the model predicted price values
predictions = model(val_input)
predictions = scaler.inverse_transform(predictions.cpu().detach().numpy())
# plot the stock price
train = data[:training_data_len]
valid = data[training_data_len:].copy()
valid["Predictions"] = predictions

mlflow.pytorch.log_model(
    registered_model_name = "ResNLS",
    artifact_path = "ResNLS",
    pytorch_model = model,
    input_example = val_input.cpu().detach().numpy(),
    signature = infer_signature(val_input.cpu().detach().numpy(), predictions)
)

y = np.array(valid["Close"])
y_hat = np.array(valid["Predictions"])
mae = metrics.mean_absolute_error(y_hat, y)
mse = metrics.mean_squared_error(y_hat, y)
rmse = metrics.mean_squared_error(y_hat, y) ** 0.5

mlflow.log_metrics({
    "mae": mae,
    "mse": mse,
    "rmse": rmse
})

mlflow.end_run()
print("MAE:{:.2F}   MSE: {:.2f}   RMSE:{:.2F}".format(mae, mse, rmse))

Successfully registered model 'ResNLS'.


MAE:25.57   MSE: 1191.36   RMSE:34.52


Created version '1' of model 'ResNLS'.
